### Homework：替换以上示例中的模型，对比不同模型在相同任务上的性能表现

在 Hugging Face Models 中找到适合你的模型：https://huggingface.co/models

#### 1.设置环境变量，用于保存模型；设置网络代理

In [1]:
import os

os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/huggingface/hub'

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

#### 1.对比文本分类任务
构造测试用数据，设置多个比对模型，依次调用，测试文本分类，分别对比各模型的实际效果。模型抽取了排名靠前的若干模型。

In [4]:
chi_texts = ['今天是一个阳光明媚的日子。',
 '这部电影太令人失望了，情节乏味，表演也很差。',
 '我喜欢和朋友一起度过愉快的时光。',
 '这家餐厅的食物味道很美味，服务也很周到。',
 '今天的工作任务很繁重，但我感到很充实。',
 '这本书的内容很有启发性，让我受益匪浅。',
 '我觉得这个问题很难回答，需要更多的思考。',
 '这个城市的环境优美，适合居住。',
 '对于这个方案，我持保留态度。',
 '听到这个好消息，我感到非常兴奋。']
eng_texts = ['Today is a sunny day.',
 'This movie is so disappointing, the plot is dull, and the performance is poor.',
 'I enjoy spending happy time with friends.',
 'The food in this restaurant is delicious, and the service is attentive.',
 "Today's workload is heavy, but I feel fulfilled.",
 'The content of this book is very inspiring and enlightening.',
 'I find this question difficult to answer and requires more thought.',
 'The environment of this city is beautiful and suitable for living.',
 'I am reserved about this plan.',
 'I am very excited to hear this good news.']
senti_labels = ['Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive']

In [5]:
from transformers import pipeline

models_list = [ {'model_name':'uer/roberta-base-finetuned-dianping-chinese','lang':'中文'},
                {'model_name':'ProsusAI/finbert','lang':'英文'},
                {'model_name':'lxyuan/distilbert-base-multilingual-cased-sentiments-student','lang':'多语言'}
               ]

for model in models_list:
    pipe = pipeline("sentiment-analysis",model=model['model_name'])
    chi_reslut = [ret for ret in pipe(chi_text for chi_text in chi_texts)]
    eng_result = [ret for ret in pipe(eng_text for eng_text in eng_texts)]
    print(model['model_name'],'模型支持语言:',model['lang'],
          '\n标签  ：',senti_labels,
          '\n中文识别：',[result['label'] for result in chi_reslut],
          '\n预测概率：',[result['score'] for result in chi_reslut],
          '\n英文识别：',[result['label'] for result in eng_result],
          '\n预测概率：',[result['score'] for result in eng_result])

/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


uer/roberta-base-finetuned-dianping-chinese 模型支持语言: 中文 
标签  ： ['Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive'] 
中文识别： ['positive (stars 4 and 5)', 'negative (stars 1, 2 and 3)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'negative (stars 1, 2 and 3)', 'positive (stars 4 and 5)', 'negative (stars 1, 2 and 3)', 'positive (stars 4 and 5)'] 
预测概率： [0.7619587182998657, 0.9963312745094299, 0.7475177645683289, 0.9657039642333984, 0.8325814604759216, 0.913057804107666, 0.6812158226966858, 0.889066219329834, 0.9668331742286682, 0.5953904390335083] 
英文识别： ['positive (stars 4 and 5)', 'negative (stars 1, 2 and 3)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'positive (stars 4 and 5)', 'negative (stars 1, 2 and 3)', 'positive (stars 4 and 5)'] 
预测概率： [0.971316635608

#### 2.NER任务

In [19]:
classifier = pipeline(task="ner", model='google/flan-t5-base',grouped_entities=True)
classifier("Hugging Face is a French company based in New York City.")

config.json: 100%|██████████| 1.40k/1.40k [00:00<00:00, 4.28MB/s]
model.safetensors: 100%|██████████| 990M/990M [02:28<00:00, 6.65MB/s] 
Some weights of T5ForTokenClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 6.48MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 14.9MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:01<00:00, 1.68MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<00:00, 7.05MB/s]
/root/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'LABEL_1',
  'score': 0.7952455,
  'word': 'Hugging Face is',
  'start': 0,
  'end': 15},
 {'entity_group': 'LABEL_0',
  'score': 0.9329552,
  'word': '',
  'start': 15,
  'end': 16},
 {'entity_group': 'LABEL_1',
  'score': 0.9491247,
  'word': 'a',
  'start': 16,
  'end': 17},
 {'entity_group': 'LABEL_0',
  'score': 0.8633172,
  'word': 'French company ',
  'start': 17,
  'end': 33},
 {'entity_group': 'LABEL_1',
  'score': 0.8657277,
  'word': 'based in New',
  'start': 33,
  'end': 45},
 {'entity_group': 'LABEL_0',
  'score': 0.9488959,
  'word': 'York City.',
  'start': 45,
  'end': 56}]

#### 3.对比问答任务
构造测试用数据，设置多个比对模型，依次调用，测试问答效果，分别对比各模型的实际效果。模型抽取了排名靠前的若干模型。

In [15]:
qas = [
    {
        'question': 'What are the advantages of using deep learning?',
        'context': 'Deep learning offers advantages such as automatic feature extraction, scalability to large datasets, and state-of-the-art performance in various tasks.'
    },
    {
        'question': 'How do recurrent neural networks (RNNs) differ from feedforward neural networks?',
        'context': 'RNNs have feedback connections that allow them to maintain a state or memory over time, making them suitable for sequential data processing tasks.'
    },
    {
        'question': 'What is the purpose of dropout in neural networks?',
        'context': 'Dropout is a regularization technique used in neural networks to prevent overfitting by randomly deactivating neurons during training.'
    },
    {
        'question': 'What are the challenges of training deep neural networks?',
        'context': 'Challenges of training deep neural networks include vanishing or exploding gradients, overfitting, and the need for large amounts of labeled data.'
    },
    {
        'question': 'How does the backpropagation algorithm work in neural networks?',
        'context': 'Backpropagation is an algorithm used to train neural networks by iteratively adjusting the weights based on the gradient of the loss function with respect to the network parameters.'
    },
    {
        'question': 'What is the role of convolutional layers in convolutional neural networks (CNNs)?',
        'context': 'Convolutional layers in CNNs perform feature extraction by applying filters to input data, capturing spatial patterns in images or other structured data.'
    },
    {
        'question': 'How does batch normalization improve training in neural networks?',
        'context': 'Batch normalization normalizes the activations of each layer in a neural network, reducing internal covariate shift and allowing for faster and more stable training.'
    },
    {
        'question': 'What are the different types of activation functions used in neural networks?',
        'context': 'Common activation functions in neural networks include sigmoid, tanh, ReLU, and softmax, each serving different purposes in network architectures.'
    },
    {
        'question': 'How does attention mechanism work in deep learning models?',
        'context': 'Attention mechanism allows models to focus on relevant parts of the input sequence, enhancing performance in tasks such as machine translation and image captioning.'
    },
    {
        'question': 'What is the role of the loss function in training neural networks?',
        'context': 'The loss function measures the difference between predicted and actual values, providing feedback to the network during training to adjust the model parameters.'
    }
]

# 单独的答案列表
answers = [
    'Automatic feature extraction, scalability, state-of-the-art performance',
    'RNNs have feedback connections, suitable for sequential data processing',
    'Prevent overfitting by deactivating neurons randomly',
    'Vanishing or exploding gradients, overfitting, need for large labeled data',
    'Iteratively adjusting weights based on gradient of loss function',
    'Perform feature extraction by applying filters to input data',
    'Normalize activations of each layer, reduce internal covariate shift',
    'Sigmoid, tanh, ReLU, softmax',
    'Focus on relevant parts of input sequence',
    'Measure difference between predicted and actual values'
]


In [17]:
from transformers import pipeline

models_list = [ {'model_name':'deepset/roberta-base-squad2','tag':'趋势排行榜第一'},
                {'model_name':'distilbert/distilbert-base-cased-distilled-squad','tag':'下载量第二'},
               ]

for model in models_list:
    
    pipe = pipeline("question-answering",model=model['model_name'])
    
    ans = [ans for ans in pipe(qa for qa in qas)]

    print(model['model_name'])
    print(answers)
    print([a['answer'] for a in ans])
    print([a['score'] for a in ans])


deepset/roberta-base-squad2
['Automatic feature extraction, scalability, state-of-the-art performance', 'RNNs have feedback connections, suitable for sequential data processing', 'Prevent overfitting by deactivating neurons randomly', 'Vanishing or exploding gradients, overfitting, need for large labeled data', 'Iteratively adjusting weights based on gradient of loss function', 'Perform feature extraction by applying filters to input data', 'Normalize activations of each layer, reduce internal covariate shift', 'Sigmoid, tanh, ReLU, softmax', 'Focus on relevant parts of input sequence', 'Measure difference between predicted and actual values']
['automatic feature extraction, scalability to large datasets', 'feedback connections', 'to prevent overfitting', 'vanishing or exploding gradients, overfitting', 'by iteratively adjusting the weights based on the gradient of the loss function', 'perform feature extraction', 'faster and more stable training', 'sigmoid, tanh, ReLU, and softmax', '

#### 任务4 总结

In [36]:
from transformers import pipeline

chi_texts = '故事背景是一个小村庄，生活着一只名叫Whiskers的调皮猫。Whiskers不同于其他猫，他有一种冒险精神和无法满足的好奇心。一个阳光明媚的下午，他发现村民们在准备野餐，他跟着偷偷尾随。村民们在河边野餐，Whiskers看着美味的食物，忍不住偷了一块三明治。但是被一阵风吹走，飞到了河里。Whiskers毫不犹豫地跳了进去，不顾危险地想要拯救三明治。他不断地向前游，终于抓住了三明治。但是他被水冲走了，被村民救了出来。村民为他的勇敢欢呼，尊重他。Whiskers虽然失去了三明治，但赢得了村民的尊敬。'

eng_texts = 'Once upon a time, in a small village nestled between rolling hills, there lived a mischievous cat named Whiskers. Whiskers was unlike any other cat in the village. He had a knack for adventure and an insatiable curiosity that often led him into trouble.One sunny afternoon, Whiskers spotted a group of villagers preparing for a picnic by the tranquil river that flowed through the heart of the village. Unable to resist the temptation of delicious food, Whiskers stealthily followed them, his tail twitching with excitement.As the villagers settled down for their picnic, Whiskers watched from a distance, his eyes fixated on the mouthwatering spread of sandwiches, fruits, and pastries. Unable to control his hunger any longer, Whiskers sprang into action, darting forward to snatch a sandwich from the edge of the blanket.But just as he was about to sink his teeth into the tasty treat, a mischievous gust of wind swept through the picnic area, carrying the sandwich away and sending it tumbling into the river. Without hesitation, Whiskers leaped into action, determined to retrieve his stolen prize.As he paddled furiously against the current, Whiskers realized that he had underestimated the power of the river. The currents were strong, and the sandwich seemed to be moving further away with each passing moment. But Whiskers refused to give up. With a burst of determination, he pushed himself harder, his paws slicing through the water as he chased after the elusive sandwich.After what felt like an eternity, Whiskers finally caught up to the sandwich, his heart pounding with triumph. But just as he reached out to grab it, he felt a pair of strong hands scoop him out of the water. It was the villagers, who had seen his daring rescue mission and rushed to save him.As Whiskers lay panting on the riverbank, the villagers laughed and cheered, amazed by his bravery. And though the sandwich was lost forever, Whiskers had gained something far more valuable – the respect and admiration of the entire village.'

models_list = [ {'model_name':'sshleifer/distilbart-cnn-12-6','tag':'下载榜第二'},
                {'model_name':'philschmid/bart-large-cnn-samsum','tag':'下载量第三'},
               ]

for model in models_list:
    
    pipe =  pipeline(task="summarization",
                      model=model['model_name'],
                      min_length=8,
                      max_length=100,)
    
    summary = [pipe(chi_texts),pipe(eng_texts)]

    print(model['model_name'])
    print(summary)

sshleifer/distilbart-cnn-12-6
[[{'summary_text': ' Whiskers: "Whiskers" \xa0- \xa0’�:\xa0‘I’ll always be happy to have your friends with me.’��'}], [{'summary_text': ' Whiskers chased a sandwich away from a picnic table by a river in a small village . The cat was about to sink his teeth into the tasty treat, but a gust of wind swept through the picnic area, sending it tumbling into the river .'}]]
philschmid/bart-large-cnn-samsum
[[{'summary_text': 'Whiskers are cute little animals.'}], [{'summary_text': 'Whiskers the cat lives in a small village. One day he saw a group of villagers preparing for a picnic by the river. He followed them and snatched a sandwich from the edge of the blanket, but a gust of wind knocked it into the river and he had to fight the current to retrieve it. The villagers rescued him.'}]]


#### 任务5.音频分类

In [4]:
from transformers import pipeline


models_list = [ {'model_name':'MIT/ast-finetuned-audioset-10-10-0.4593','tag':'下载量第一'},
                {'model_name':'alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech','tag':'下载量第3'},
               ]


for model in models_list:
    
    pipe =  pipeline(task="audio-classification",
                      model=model['model_name'])
    
    preds = pipe("data/audio/mlk.flac")

    print(model['model_name'],preds)


/root/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/audio_utils.py:198: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


MIT/ast-finetuned-audioset-10-10-0.4593 [{'score': 0.4207741916179657, 'label': 'Speech'}, {'score': 0.1793096661567688, 'label': 'Rain on surface'}, {'score': 0.13007479906082153, 'label': 'Rain'}, {'score': 0.09596990048885345, 'label': 'Raindrop'}, {'score': 0.057824958115816116, 'label': 'Music'}]


config.json: 100%|██████████| 2.19k/2.19k [00:00<00:00, 6.87MB/s]
model.safetensors: 100%|██████████| 1.26G/1.26G [05:20<00:00, 3.94MB/s]
Some weights of the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkp

alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech [{'score': 0.9985793828964233, 'label': 'male'}, {'score': 0.0014206045307219028, 'label': 'female'}]


#### 6.语音识别转文本

In [8]:
from transformers import pipeline


models_list = [ {'model_name':'openai/whisper-tiny','tag':'whisper'},
               ]


for model in models_list:
    
    pipe =  pipeline(task="automatic-speech-recognition",
                      model=model['model_name'])
    
    text = pipe("data/audio/mlk.flac")

    print(model['model_name'],text)


config.json: 100%|██████████| 1.98k/1.98k [00:00<00:00, 5.66MB/s]
model.safetensors: 100%|██████████| 151M/151M [00:58<00:00, 2.60MB/s] 
generation_config.json: 100%|██████████| 3.75k/3.75k [00:00<00:00, 10.8MB/s]
tokenizer_config.json: 100%|██████████| 805/805 [00:00<00:00, 2.24MB/s]
vocab.json: 100%|██████████| 836k/836k [00:01<00:00, 598kB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 3.63MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 3.60MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 286kB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 48.6MB/s]
special_tokens_map.json: 100%|██████████| 2.08k/2.08k [00:00<00:00, 5.51MB/s]
preprocessor_config.json: 100%|██████████| 185k/185k [00:01<00:00, 106kB/s]
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might b

openai/whisper-tiny {'text': ' I have a dream. Good one day. This nation will rise up. Live out the true meaning of its dream.'}


#### 6.图像分类

In [11]:
from transformers import pipeline


models_list = [ {'model_name':'microsoft/resnet-50','tag':'mostlike2'},
                {'model_name':'apple/mobilevit-small','tag':'mostlike4'},
               ]
for model in models_list:

    classifier = pipeline(task="image-classification",model=model['model_name'])

    preds = classifier(
        "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
    )
    preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
    print(*preds, sep="\n")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


{'score': 0.5874, 'label': 'lynx, catamount'}
{'score': 0.1289, 'label': 'tabby, tabby cat'}
{'score': 0.075, 'label': 'marmot'}
{'score': 0.0382, 'label': 'badger'}
{'score': 0.0131, 'label': 'Egyptian cat'}


config.json: 100%|██████████| 70.0k/70.0k [00:00<00:00, 118kB/s]
pytorch_model.bin: 100%|██████████| 22.5M/22.5M [00:09<00:00, 2.27MB/s]
preprocessor_config.json: 100%|██████████| 187/187 [00:00<00:00, 579kB/s]
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


{'score': 0.9441, 'label': 'lynx, catamount'}
{'score': 0.0041, 'label': 'tiger cat'}
{'score': 0.0036, 'label': 'grey fox, gray fox, Urocyon cinereoargenteus'}
{'score': 0.0026, 'label': 'Egyptian cat'}
{'score': 0.002, 'label': 'tabby, tabby cat'}


#### 7.目标检测

In [12]:
!pip install timm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [23]:
from transformers import pipeline

detector = pipeline(task="object-detection",model='hustvl/yolos-tiny')

preds = detector(
    "data/image/cat_dog.jpg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"], "box": pred["box"]} for pred in preds]
preds

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


[{'score': 0.9946,
  'label': 'cat',
  'box': {'xmin': 75, 'ymin': 60, 'xmax': 290, 'ymax': 369}},
 {'score': 0.9899,
  'label': 'dog',
  'box': {'xmin': 280, 'ymin': 18, 'xmax': 479, 'ymax': 416}}]